In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
!pip install adabelief-pytorch==0.2.0

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
#!pip install nbformat # 설치 시도 
!pip install -U nbformat # 기설치시, update 

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 77 kB 18.5 MB/s eta 0:00:01
     |████████████████████████████████| 116 kB 41.6 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  NOTE: The current PATH contains path(s) starting with `~`, which may not be expanded by all applications.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.5.1 requires jupyter-client>=6.1.5, but you have jupyter-client 6.0.0 which is incompatible.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
import os
import re
import numpy as np
from tqdm import tqdm
from glob import glob
import json
import requests
import tensorflow as tf
from transformers import BertModel, TFBertModel, TFRobertaModel, RobertaTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel, AutoModelForSequenceClassification
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold,StratifiedKFold
from adabelief_pytorch import AdaBelief
from transformers.optimization import get_cosine_schedule_with_warmup
from tqdm import tqdm, tqdm_notebook
import shutil
import gc
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

"klue/roberta-large" 사용 시 "klue/roberta-large" 임베딩과 NUM_EPOCHS = 3

"monologg/kobigbird-bert-base" 모델 사용시  "monologg/kobigbird-bert-base" 임베딩으로  NUM_EPOCHS = 6

In [2]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)
BATCH_SIZE = 32
NUM_EPOCHS = 3

L_RATE = 1e-5
MAX_LEN = 128
max_grad_norm=1
log_interval=200
NUM_CORES = os.cpu_count()
device = torch.device("cuda:0")

DATA_IN_PATH = './data'
DATA_OUT_PATH = "./data"

In [54]:
#두가지 임베딩
#"klue/roberta-large"
#"monologg/kobigbird-bert-base"
#tokenizer = AutoTokenizer.from_pretrained("monologg/kobigbird-bert-base", cache_dir='bert_ckpt', do_lower_case=False)
tokenizer = AutoTokenizer.from_pretrained("klue/roberta-large", cache_dir='bert_ckpt', do_lower_case=False)


Downloading:   0%|          | 0.00/459 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/470k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/56.0 [00:00<?, ?B/s]

In [55]:
PATH = 'numpy값new'

In [56]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
submission = pd.read_csv('./data/sample_submission.csv')

In [57]:
train.문장

0                    0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.
1        이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...
2        정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...
3        서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...
4                 익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.
                               ...                        
16536    ＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...
16537    수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...
16538    김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...
16539    1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...
16540                                        《목민심서》의 내용이다.
Name: 문장, Length: 16541, dtype: object

In [58]:


# 2. Label Encoding (유형, 극성, 시제, 확실성)
from sklearn.preprocessing import LabelEncoder
type_le = LabelEncoder()
train["유형"] = type_le.fit_transform(train["유형"].values)


polarity_le = LabelEncoder()
train["극성"] = polarity_le.fit_transform(train["극성"].values)


tense_le = LabelEncoder()
train["시제"] = tense_le.fit_transform(train["시제"].values)


certainty_le = LabelEncoder()
train["확실성"] = certainty_le.fit_transform(train["확실성"].values)


In [59]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, '문장']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True, 
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        target = torch.tensor(self.df_data.loc[index, "유형"])
        sample = (padded_token_list, token_type_id , att_mask, target)
        return sample
    def __len__(self):
        return len(self.df_data)

In [60]:
class TestDataset(Dataset):
    def __init__(self, df):
        self.df_data = df
    def __getitem__(self, index):
        # get the sentence from the dataframe
        sentence = self.df_data.loc[index, '문장']
        encoded_dict = tokenizer(
          text = sentence,
          add_special_tokens = True, 
          max_length = MAX_LEN,
          pad_to_max_length = True,
          truncation=True,           # Pad & truncate all sentences.
          return_tensors="pt")

        padded_token_list = encoded_dict['input_ids'][0]
        token_type_id = encoded_dict['token_type_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        sample = (padded_token_list, token_type_id , att_mask)
        return sample
    def __len__(self):
        return len(self.df_data)

In [61]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

h값으로 미세조정한 후 저장
ex) Roberta229 = h값 229

In [62]:
#AUto 
train_data = TrainDataset(train)

test_data = TestDataset(test)

train_dataloader = torch.utils.data.DataLoader(train_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True,
                                      num_workers=NUM_CORES)
test_dataloader = torch.utils.data.DataLoader(test_data,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False,
                                      num_workers=NUM_CORES)
#두가지 모델
#"klue/roberta-large"
#"monologg/kobigbird-bert-base"
#kobigbird-bert-base
#model = AutoModelForSequenceClassification.from_pretrained("skt/kogpt2-base-v2",um_labels=3)
model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large",num_labels=3)



####미세조정
n=0
for name, child in model.named_children():
    if n==0:
      h=0
      for param in child.parameters():
        if h<=0: #이부분 숫자 조절로 fine-tuning => Roberta229: h=229
          param.requires_grad = Falsex
        h+=1
    n+=1
#####
    # print(param)
model.to(device)
optimizer = AdaBelief(model.parameters(), lr=1e-5, eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False)

warmup_ratio = 0.1
t_total = len(train_dataloader) * NUM_EPOCHS
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

for e in range(NUM_EPOCHS):
    train_acc = 0.0
    test_acc = 0.0
    best_acc =0.0
    model.train()
    torch.set_grad_enabled(True)
    for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.to(device)
        outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
        loss = outputs[0]
        out = outputs[1]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        _, preds = torch.max(out, dim=1)
        train_acc += f1_score(preds.cpu(), label.cpu(),average='weighted')
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

preds = [] 
model.eval()
torch.set_grad_enabled(False)
for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(test_dataloader)):
    input_id = input_id.long().to(device)
    token_type_id = token_type_id.long().to(device)
    attention_mask = attention_mask.long().to(device)
    outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
    out = outputs[0]
    for inp in out:
      preds.append(inp.detach().cpu().numpy())
Preds = np.array(preds)

Downloading:   0%|          | 0.00/459 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/534M [00:00<?, ?B/s]

Some weights of the model checkpoint at tunib/electra-ko-en-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at tunib/electra-ko-en-base and are newly initialized: ['classifier.dense.weight', 'cla

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


<ipython-input-62-8509416f4710>:49: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm_notebook(train_dataloader)):


...

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

epoch 3 batch id 1 loss 0.6967983841896057 train acc 0.896551724137931
epoch 3 batch id 201 loss 0.7639550566673279 train acc 0.9020066196778973
epoch 3 batch id 401 loss 0.5516735315322876 train acc 0.9017060392009971
epoch 3 train acc 0.8994747713897175


<ipython-input-62-8509416f4710>:71: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/222 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

In [26]:
#332 : epoch 3 train acc 0.9187989758050054
# 273 : epoch 3 train acc 0.9268683707635386
# 325 : epoch 3 train acc 0.9205169095085536
# 251 : epoch 3 train acc 0.9313042428174262
# kobig120 : epoch 3 train acc 0.9093886905876433

#극성
#332 : epoch 3 train acc 0.9854487346754297
#273 : epoch 3 train acc 0.9868469425354284
#325 : epoch 3 train acc 0.984686812931019
#251 : epoch 3 train acc 0.988702150120287
# kobig120 : epoch 3 train acc 0.9812677962160321
#시제
#332 : epoch 3 train acc 0.919275517196581
#273 : epoch 3 train acc 0.9312424963644657
#325 : epoch 3 train acc 0.9180134893123439
#251 : epoch 3 train acc 0.9349046320802006
#

#확실성
# 332 : epoch 3 train acc 0.9496036063037623
# 273 : epoch 3 train acc 0.9542198576228881
# 325 : epoch 3 train acc 0.9492931109324948
# 251 : epoch 3 train acc 0.9584442313345445

#유형
# 332 epoch 3 train acc 0.9121554583652488
# 273 epoch 3 train acc 0.949908789569253
# 325 epoch 3 train acc 0.9160459562169243
# 251 epoch 3 train acc 0.9580608916106587

#시제
# 332 epoch 3 train acc 0.8984469604539396
# 273 epoch 3 train acc 0.9372775441399579
#극성
#epoch 3 train acc 0.9770698392382055
#epoch 3 train acc 0.992289766970767
#epoch 3 train acc 0.9792922031080108
#epoch 3 train acc 0.9941761801934311

num_list=[332,273,325,251]
for i in num_list:
    #AUto 
    train_data = TrainDataset(train)

    test_data = TestDataset(test)

    train_dataloader = torch.utils.data.DataLoader(train_data,
                                            batch_size=BATCH_SIZE,
                                            shuffle=True,
                                          num_workers=NUM_CORES)
    test_dataloader = torch.utils.data.DataLoader(test_data,
                                            batch_size=BATCH_SIZE,
                                            shuffle=False,
                                          num_workers=NUM_CORES)
    #두가지 모델
    #"klue/roberta-large"
    #"monologg/koelectra-base-v3-discriminator"
    #kobigbird-bert-base
    #model = AutoModelForSequenceClassification.from_pretrained("#"monologg/kobigbird-bert-base"",um_labels=3)
    model = AutoModelForSequenceClassification.from_pretrained("klue/roberta-large",num_labels=2)

    ####미세조정
    n=0
    for name, child in model.named_children():
        if n==0:
          h=0
          for param in child.parameters():
            if h<=229: #이부분 숫자 조절로 fine-tuning => Roberta229: h=229
              param.requires_grad = False
            h+=1
        n+=1
    #####
        # print(param)
    model.to(device)
    optimizer = AdaBelief(model.parameters(), lr=1e-5, eps=1e-16, betas=(0.9,0.999), weight_decouple = True, rectify = False)

    warmup_ratio = 0.1
    t_total = len(train_dataloader) * NUM_EPOCHS
    warmup_step = int(t_total * warmup_ratio)
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

    for e in range(NUM_EPOCHS):
        train_acc = 0.0
        test_acc = 0.0
        best_acc =0.0
        model.train()
        torch.set_grad_enabled(True)
        for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm_notebook(train_dataloader)):
            optimizer.zero_grad()
            input_id = input_id.long().to(device)
            token_type_id = token_type_id.long().to(device)
            attention_mask = attention_mask.long().to(device)
            label = label.to(device)
            outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask, labels=label)
            loss = outputs[0]
            out = outputs[1]
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()  # Update learning rate schedule
            _, preds = torch.max(out, dim=1)
            train_acc += f1_score(preds.cpu(), label.cpu(),average='weighted')
            if batch_id % log_interval == 0:
                print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
        print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    preds = [] 
    model.eval()
    torch.set_grad_enabled(False)
    for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(test_dataloader)):
        input_id = input_id.long().to(device)
        token_type_id = token_type_id.long().to(device)
        attention_mask = attention_mask.long().to(device)
        outputs = model(input_ids=input_id, token_type_ids=token_type_id, attention_mask=attention_mask)
        out = outputs[0]
        for inp in out:
          preds.append(inp.detach().cpu().numpy())
    Preds = np.array(preds) 
    np.save(PATH+'Roberta'+str(i)+'_certainty_j.npy',arr=Preds)
    

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifi

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief


<ipython-input-26-24cf2222f175>:90: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (input_id,token_type_id,attention_mask,label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1346 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

...

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

epoch 3 batch id 1 loss 0.033448439091444016 train acc 1.0
epoch 3 batch id 201 loss 0.07768499106168747 train acc 0.9639328926007862
epoch 3 batch id 401 loss 0.006930421572178602 train acc 0.9630635529953346
epoch 3 batch id 601 loss 0.017165614292025566 train acc 0.9624684774128149
epoch 3 batch id 801 loss 0.167031392455101 train acc 0.9612577527090564
epoch 3 batch id 1001 loss 0.016067493706941605 train acc 0.9616986532743238
epoch 3 batch id 1201 loss 0.20379367470741272 train acc 0.9625332442678458
epoch 3 train acc 0.9631817653877773


<ipython-input-26-24cf2222f175>:112: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (input_id,token_type_id,attention_mask) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/222 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2173: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input si

In [25]:
results = np.argmax(Preds, axis=1)
#submission['topic_idx']= results

In [26]:
test_pred=type_le.inverse_transform(results)
#polarity_pred=polarity_le.inverse_transform(results)
#tense_pred=tense_le.inverse_transform(results)
#certainty_le_pred=certainty_le.inverse_transform(results)

In [27]:
submission['유형']= test_pred

In [35]:
submission.유형.value_counts()

사실형    5935
추론형     825
대화형     241
예측형      89
Name: 유형, dtype: int64

In [100]:
submission['극성']= polarity_pred

In [36]:
submission.극성.value_counts()

긍정    6799
부정     248
미정      43
Name: 극성, dtype: int64

In [78]:
submission['시제']= tense_pred

In [37]:
submission.시제.value_counts()

과거    3434
현재    2976
미래     680
Name: 시제, dtype: int64

In [81]:
submission['확실성']= certainty_le_pred

In [38]:
submission.확실성.value_counts()

확실     6655
불확실     435
Name: 확실성, dtype: int64

In [87]:
submission['label']=submission.유형+'-'+submission.극성+'-'+submission.시제+'-'+submission.확실성

In [89]:
last=submission[['ID','label']]
last=last.set_index('ID')
last.to_csv('klue_최종label.csv',encoding='utf-8-sig',)

In [23]:
Roberta332_type_j=Preds 
np.save(PATH+'Roberta332_poriarty_j.npy',arr=Roberta332_type_j)

In [31]:
Roberta273_type_j=Preds 
np.save(PATH+'Roberta273_type_j.npy',arr=Roberta273_type_j)

In [34]:
Roberta325_type_j=Preds 
np.save(PATH+'Roberta325_type_j.npy',arr=Roberta325_type_j)

In [13]:
Roberta251_type_j=Preds 
np.save(PATH+'Roberta251_type_j.npy',arr=Roberta251_type_j)

In [11]:
Roberta332_certainty=Preds 
np.save(PATH+'Roberta332_certainty.npy',arr=Roberta332_certainty)

In [18]:
Roberta273_certainty=Preds 
np.save(PATH+'Roberta273_certainty.npy',arr=Roberta273_certainty)

In [38]:
Roberta325_tense=Preds 
np.save(PATH+'Roberta325_tense.npy',arr=Roberta325_tense)

In [14]:
Roberta251_certainty=Preds
np.save(PATH+'Roberta251_certainty.npy',arr=Roberta251_certainty)

In [69]:
kobig120=Preds 
np.save(PATH+'Electra120.npy',arr=kobig120)

In [ ]:
#332 : epoch 3 train acc 0.9187989758050054
# 273 : epoch 3 train acc 0.9268683707635386
# 325 : epoch 3 train acc 0.9205169095085536
# 251 : epoch 3 train acc 0.9313042428174262
# kobig120 : epoch 3 train acc 0.9093886905876433

#극성
#332 : epoch 3 train acc 0.9854487346754297
#273 : epoch 3 train acc 0.9868469425354284
#325 : epoch 3 train acc 0.984686812931019
#251 : epoch 3 train acc 0.988702150120287
# kobig120 : epoch 3 train acc 0.9812677962160321
#시제
#332 : epoch 3 train acc 0.919275517196581
#273 : epoch 3 train acc 0.9312424963644657
#325 : epoch 3 train acc 0.9180134893123439
#251 : epoch 3 train acc 0.9349046320802006
#

#확실성
# 332 : epoch 3 train acc 0.9496036063037623
# 273 : epoch 3 train acc 0.9539584020942307
# 325 : epoch 3 train acc 0.9492931109324948
# 251 : epoch 3 train acc 0.9584442313345445

In [25]:
kobig120=np.load('numpy값지평kobig120_type_j.npy')
Roberta251=np.load('numpy값지평Roberta251_type_j.npy')
Roberta273=np.load('numpy값지평Roberta273_type_j.npy')
Roberta325=np.load('numpy값지평Roberta325_type_j.npy')
Roberta332=np.load('numpy값지평Roberta332_type_j.npy')

Roberta332_polarity= np.load('numpy값지평Roberta332_poriarty_j.npy')
Roberta325_polarity= np.load('numpy값지평Roberta325_polarity_j.npy')
Roberta273_polarity= np.load('numpy값지평Roberta273_polarity_j.npy')
Roberta251_polarity= np.load('numpy값지평Roberta251_polarity_j.npy')
kobig120_polarity=np.load('numpy값지평kobig120_polarity_j.npy')

Roberta332_tense= np.load('numpy값지평Roberta332_tense_j.npy')
Roberta325_tense= np.load('numpy값지평Roberta325_tense_j.npy')
Roberta273_tense= np.load('numpy값지평Roberta273_tense_j.npy')
Roberta251_tense= np.load('numpy값지평Roberta251_tense_j.npy')
kobig120_tense=np.load('numpy값지평kobig120_tense_j.npy')

Roberta332_certainty= np.load('numpy값지평Roberta332_certainty_j.npy')
Roberta325_certainty= np.load('numpy값지평Roberta325_certainty_j.npy')
Roberta273_certainty= np.load('numpy값지평Roberta273_certainty_j.npy')
Roberta251_certainty= np.load('numpy값지평Roberta251_certainty_j.npy')
kobig120_certainty=np.load('numpy값지평kobig120_certainty_j.npy')


In [4]:
kobig120=np.load('numpy값Electra120.npy')
Roberta251=np.load('numpy값Roberta251.npy')
Roberta273=np.load('numpy값Roberta273.npy')
Roberta325=np.load('numpy값Roberta325.npy')
Roberta332=np.load('numpy값Roberta332_type.npy')

Roberta332_polarity= np.load('numpy값Roberta332_polarity.npy')
Roberta325_polarity= np.load('numpy값Roberta325_polarity.npy')
Roberta273_polarity= np.load('numpy값Roberta273_polarity.npy')
Roberta251_polarity= np.load('numpy값Roberta251_polarirty.npy')
kobig120_polarity=np.load('numpy값kobig120_polarity.npy')

Roberta332_tense= np.load('numpy값Roberta332_tense.npy')
Roberta325_tense= np.load('numpy값Roberta325_tense.npy')
Roberta273_tense= np.load('numpy값Roberta273_tense.npy')
Roberta251_tense= np.load('numpy값Roberta251_tense.npy')
kobig120_tense=np.load('numpy값kobig120_tense.npy')

Roberta332_certainty= np.load('numpy값Roberta332_certainty.npy')
Roberta325_certainty= np.load('numpy값Roberta325_certainty.npy')
Roberta273_certainty= np.load('numpy값Roberta273_certainty.npy')
Roberta251_certainty= np.load('numpy값Roberta251_certainty.npy')
kobig120_certainty=np.load('numpy값kobig120_certainty.npy')


In [20]:
#score 0.75209
Pred_values_type = Roberta332*0.1 + Roberta325 * 0.15 + Roberta273 * 0.15 + Roberta251 * 0.4 + kobig120 * 0.2
Pred_values_polarity = Roberta332_polarity*0.1 + Roberta325_polarity * 0.15 + Roberta273_polarity * 0.15 + Roberta251_polarity * 0.4 + kobig120_polarity * 0.2
Pred_values_tense = Roberta332_tense*0.1 + Roberta325_tense * 0.15 + Roberta273_tense * 0.15 + Roberta251_tense * 0.4 + kobig120_tense * 0.2
Pred_values_certainty = Roberta332_certainty*0.1 + Roberta325_certainty * 0.15 + Roberta273_certainty * 0.15 + Roberta251_certainty * 0.4 + kobig120_certainty * 0.2

In [32]:
results_type = np.argmax(Pred_values_type, axis=1)
type_pred_ang=type_le.inverse_transform(results_type)
submission['유형']= type_pred_ang

results_polarity = np.argmax(Pred_values_polarity, axis=1)
polarity_pred_ang=polarity_le.inverse_transform(results_polarity)
submission['극성']= polarity_pred_ang

results_tense = np.argmax(Pred_values_tense, axis=1)
tense_pred_ang=tense_le.inverse_transform(results_tense)
submission['시제']= tense_pred_ang

results_certainty = np.argmax(Pred_values_certainty, axis=1)
certainty_pred_ang=certainty_le.inverse_transform(results_certainty)
submission['확실성']= certainty_pred_ang

submission['label']=submission.유형+'-'+submission.극성+'-'+submission.시제+'-'+submission.확실성
last=submission[['ID','label']]
last=last.set_index('ID')
last.to_csv('klue_kobig_ang_최종label.csv',encoding='utf-8-sig',)
#submission.to_csv(PATH + 'daconnews.csv', index=False)